In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

# A Few Shot Learning

In [3]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,👏,🚀


# 프롬프트 엔지니어링 모범 사례

## 명확한 지침 작성

예시:

-----------------------
프롬프트:

피보나치 수열을 계산하는 코드를 작성하세요.

더 나은 프롬프트:

피보나치 수열을 효율적으로 계산하는 TypeScript 함수를 작성하세요. 각 부분이 무엇을 하고 왜 그렇게 작성되었는지 설명하는 주석을 충분히 추가하세요.

----------------------

프롬프트:

회의록을 요약하세요.

더 나은 프롬프트:

회의록을 한 문단으로 요약하세요. 그런 다음 발표자와 각 발표자의 주요 요점을 나열한 마크다운 목록을 작성하세요. 마지막으로 발표자가 제안한 다음 단계나 실행 항목이 있다면 이를 나열하세요.


## 역할 놀이

예시:

-----------------------

시스템 메시지: 내가 무언가를 작성하는 데 도움을 요청하면, 당신은 모든 문단에 적어도 하나의 농담이나 장난스러운 댓글이 포함된 문서를 작성하여 응답할 것입니다.

----------------------


## 입력 텍스트 분할

예시:

------------------------

사용자 메시지: 세 개의 큰따옴표로 구분된 텍스트를 하이쿠로 요약하세요.

"""여기에 텍스트를 삽입하세요"""

------------------------

시스템 메시지: 동일한 주제에 대한 두 개의 기사(XML 태그로 구분됨)가 제공됩니다. 먼저 각 기사의 주장을 요약하세요. 그런 다음 어떤 기사가 더 나은 주장을 펼치는지와 그 이유를 설명하세요.

사용자 메시지:

\<article> 첫 번째 기사를 여기에 삽입하세요 \</article>

\<article> 두 번째 기사를 여기에 삽입하세요 \</article>

------------------------

## 작업을 완료하기 위한 단계와 프로세스 설명

예시:

--------------------------------

시스템 메시지:
다음 단계별 지침을 사용하여 사용자 입력에 응답하세요.

1단계 - 사용자가 세 개의 큰따옴표로 구분된 텍스트를 제공합니다. 이 텍스트를 "요약:"이라는 접두어와 함께 한 문장으로 요약하세요.

2단계 - 1단계의 요약을 스페인어로 번역하고 "번역:"이라는 접두어를 추가하세요.

---------------------------------

---

# 🎯 실습 과제: 프롬프트 엔지니어링으로 코드 개선하기

## 📌 과제 목표
아래 **AS-IS 코드**를 프롬프트 엔지니어링 기법을 적용하여 개선해보세요.

---

## 🔴 AS-IS: 개선 전 코드

다음은 회의록을 요약하는 간단한 챗 완성 코드입니다:

In [4]:
# 회의록 요약 코드
meeting_note = """
2025년 11월 영업 1팀 주간 회의

참석자: 김부장, 송과장, 정대리, 권사원

안건:
- Q4 목표 달성 전략
- 신규 거래처 발굴 현황
- 고객 클레임 처리 방안

김부장: 이번 달 목표 120% 달성 필수. 송과장이 대형 거래처 집중 공략.
송과장: A그룹 PT 준비 완료. 정대리가 제안서 검토 후 내일까지 최종본 제출.
정대리: B사 클레임 건 처리 완료. 권사원이 후속 관리 담당.
권사원: 신규 거래처 10곳 컨택 중. 다음주까지 미팅 일정 잡겠습니다.

다음 회의: 12월 5일 오전 9시
"""

prompt = f"다음 회의록을 요약해줘:\n\n{meeting_note}"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

**2025년 11월 영업 1팀 주간 회의 요약**

- **Q4 목표 달성 전략:** 이번 달 매출 목표 120% 달성 필수. 송과장이 대형 거래처 공략 강화.
- **신규 거래처 발굴:** 권사원, 신규 거래처 10곳 컨택 중이며 다음주까지 미팅 일정 예정.
- **고객 클레임 처리:** 정대리가 B사 클레임 건 처리 완료, 권사원이 후속 관리 담당.
- **A그룹 PT:** 송과장 PT 준비 완료, 정대리가 제안서 검토 후 내일까지 최종본 제출 예정.

**다음 회의:** 12월 5일 오전 9시


### 😢 AS-IS 코드의 문제점

1. **역할(Role) 정의 없음**
   - System 메시지가 없어 AI의 역할/전문성이 명확하지 않음
   - "당신은 회의록 요약 전문가입니다" 같은 역할 설정 부재

2. **지침이 불명확함**
   - "요약해줘"만 요청 → 어떤 형식으로? 어떤 내용을?
   - 핵심 내용, 액션 아이템, 다음 일정 등 구체적 요구 없음

3. **출력 형식이 지정되지 않음**
   - 글머리표? 문단? JSON? → 모델이 자유롭게 결정
   - 일관된 형식이 필요한 실무에서는 문제

4. **데이터와 지침이 분리되지 않음**
   - User 메시지에 지침과 회의록 내용이 섞여 있음
   - 구분자(### 또는 """)로 명확히 구분 필요

---

## ✅ 해야 할 일

위 AS-IS 코드를 아래 **4가지 프롬프트 엔지니어링 기법**을 적용하여 개선하세요:

### 1️⃣ **명확한 지침 작성**
- 출력 형식을 구체적으로 지정하세요
- 예: "다음 형식으로만 답변: 감정: [긍정/부정]"

### 2️⃣ **역할 놀이 (Role-Playing)**
- System 메시지에 AI의 역할을 명확히 정의하세요
- 예: "당신은 고객 감정 분석 전문가입니다."

### 3️⃣ **입력 텍스트 분할**
- 지침(instruction)과 데이터(data)를 명확히 구분하세요
- 구분자 사용: `"""` 또는 `###` 또는 XML 태그

### 4️⃣ **System/User 역할 분리**
- System: 역할 정의 + 출력 규칙
- User: 실제 분석할 데이터

---

## 💡 개선 예시 힌트

```python
# System 메시지: 역할과 규칙 정의
system_prompt = """당신은 회의록 요약 전문가입니다.
회의록을 분석하여 다음 형식으로 정리하세요:

📌 핵심 요약: [한 문장으로 회의 내용 요약]
👥 참석자: [참석자 목록]
✅ 주요 결정사항: [글머리표로 나열]
📋 액션 아이템: [담당자와 함께 나열]
📅 다음 일정: [날짜와 안건]
"""

# User 메시지: 데이터를 구분자로 명확히 표시
user_prompt = f"""다음 회의록을 요약하세요:

###
{meeting_note}
###
"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.3  # 일관된 요약
)

print(response.choices[0].message.content)
```

---

## 기대 효과

**AS-IS (개선 전)**
```
이 회의는 영업 1팀의 Q4 목표 달성에 관한 것입니다. 김부장, 송과장, 정대리, 권사원이 참석했으며 이번 달 목표 120% 달성을 목표로 하고 있습니다. A그룹 PT 준비가 완료되었고 신규 거래처 발굴을 진행 중입니다.
```

**TO-BE (개선 후)**
```
📌 핵심 요약: Q4 목표 120% 달성을 위한 영업 1팀 전략 회의

👥 참석자: 김부장, 송과장, 정대리, 권사원

✅ 주요 결정사항:
- 이번 달 목표 120% 달성 필수
- A그룹 대형 거래처 집중 공략
- B사 클레임 처리 완료

📋 액션 아이템:
- 송과장: A그룹 대형 거래처 공략
- 정대리: 제안서 검토 후 내일까지 최종본 제출
- 권사원: B사 후속 관리, 신규 거래처 10곳 다음주까지 미팅 일정

📅 다음 일정: 12월 5일 오전 9시
```

---

## 🔍 추가 도전 과제

1. **Few-Shot Learning 추가**
   - System 프롬프트에 회의록 요약 예시 1-2개 추가

2. **다단계 처리**
   - 1단계: 핵심 내용 추출
   - 2단계: 중요도별 분류
   - 3단계: 액션 아이템 우선순위 지정

3. **JSON 형식 출력**
   - 구조화된 JSON으로 출력하여 프로그램에서 바로 활용

---

<details>
<summary><strong>📝 샘플 답안 보기</strong> (클릭하여 펼치기/접기)</summary>

<br>

### 샘플 답안: 프롬프트 엔지니어링 적용

#### 방법 1: 기본 개선 (4가지 기법 모두 적용)

```python
meeting_note = """
2024년 11월 영업 1팀 주간 회의

참석자: 김부장, 송과장, 정대리, 권사원

안건:
- Q4 목표 달성 전략
- 신규 거래처 발굴 현황
- 고객 클레임 처리 방안

김부장: 이번 달 목표 120% 달성 필수. 송과장이 대형 거래처 집중 공략.
송과장: A그룹 PT 준비 완료. 정대리가 제안서 검토 후 내일까지 최종본 제출.
정대리: B사 클레임 건 처리 완료. 권사원이 후속 관리 담당.
권사원: 신규 거래처 10곳 컨택 중. 다음주까지 미팅 일정 잡겠습니다.

다음 회의: 12월 5일 오전 9시
"""

# 1️⃣ 역할 놀이 + 2️⃣ 명확한 지침 + 출력 형식 정의
system_prompt = """당신은 회의록 요약 전문가입니다.
회의록을 분석하여 반드시 다음 형식으로만 정리하세요:

📌 핵심 요약: [한 문장으로 회의 내용 요약]
👥 참석자: [참석자 목록]
✅ 주요 결정사항: [글머리표로 나열]
📋 액션 아이템: [담당자: 할 일 형식으로 나열]
📅 다음 일정: [날짜와 안건]

그 외 다른 내용은 출력하지 마세요."""

# 3️⃣ 입력 텍스트 분할 (구분자 사용)
user_prompt = f"""다음 회의록을 요약하세요:

###
{meeting_note}
###"""

# 4️⃣ System/User 역할 분리
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.3  # 일관된 요약
)

print(response.choices[0].message.content)
```

---

#### 방법 2: Few-Shot Learning 추가

```python
# Few-Shot Learning: 예시를 통한 학습
system_prompt = """당신은 회의록 요약 전문가입니다.

다음 예시를 참고하여 회의록을 요약하세요:

예시:
입력: "마케팅팀 회의. 참석: 홍팀장, 최대리. 안건: Q4 광고 캠페인. 홍팀장: 예산 500만원 승인. 최대리: 인스타그램 광고 집중. 다음 회의: 11/20"

출력:
📌 핵심 요약: Q4 광고 캠페인 예산 500만원 승인
👥 참석자: 홍팀장, 최대리
✅ 주요 결정사항:
- 예산 500만원 승인
- 인스타그램 광고 집중
📋 액션 아이템:
- 최대리: 인스타그램 광고 집행
📅 다음 일정: 11월 20일

이제 다음 회의록을 동일한 형식으로 요약하세요."""

user_prompt = f"""###
{meeting_note}
###"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.2
)

print(response.choices[0].message.content)
```

---

#### 방법 3: JSON 형식 출력

```python
# JSON 구조화 출력: 프로그램에서 바로 활용 가능
system_prompt = """당신은 회의록 요약 전문가입니다.
회의록을 분석하여 반드시 다음 JSON 형식으로만 출력하세요:

{
  "summary": "핵심 요약 한 문장",
  "attendees": ["참석자1", "참석자2"],
  "decisions": ["결정사항1", "결정사항2"],
  "action_items": [
    {"owner": "담당자", "task": "할 일"},
    {"owner": "담당자", "task": "할 일"}
  ],
  "next_meeting": {
    "date": "날짜",
    "time": "시간"
  }
}

JSON만 출력하고 다른 설명은 추가하지 마세요."""

user_prompt = f"""다음 회의록을 분석하세요:

###
{meeting_note}
###"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.2
)

print(response.choices[0].message.content)

# JSON 파싱 가능
import json
result = json.loads(response.choices[0].message.content)
print(f"\n담당자별 액션 아이템:")
for item in result['action_items']:
    print(f"- {item['owner']}: {item['task']}")
```

</details>

---

In [5]:
# 코드 작성